<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900"> **7:** 有状态 LLM 入门</font>

### **展望**

在之前的 notebook 中，我们试了一些仅解码器的大模型，经过足够的指令微调之后基本上能执行任何操作（如果您的指令设计的合理，并且训练数据里包括了所有必要的细节）。我们还尝试了 Llama-2，看到它强大的文本生成能力，并开始挖掘其更通用的潜力。但与所有模型一样，Llama-2 本身仍非常有限。

在此 notebook 中，我们将探究这背后的主要原因，并接触一些尝试解决随机鹦鹉（stochastic parrot）问题的框架。

#### **学习目标：**

* 探究如何用状态和外部信息来支持 LLM。
* 了解一些用大型聊天模型构建有趣且实用 LLM 应用的开源项目。

## 7.1. LLM 编排（Orchestration）简介

上一个 notebook 中，我们对经过聊天微调的 Llama-2 做了相当数量的试验，得到了一些还不错的结果。您可能还记得我们做过的事包括：

* **在不遵循任何指令格式的情况下测试模型，**模型难以适时的停止生成，不可预测，同时话也比较少。
* **使用适当的系统消息/指令格式测试模型，**模型能够在上下文学习中表现的很好，通常能够遵循指令。然而，它不能避免不必要的闲聊，并且在被要求保持固定格式的时候表现不好。
* **使用多个指令模拟对话来测试模型，**模型能在没有太多帮助的情况下很好的生成新的响应！好像模型就是被训练来做这个的。

所有这些促使 Llama-2 更通用的尝试都强调了一个显而易见的事实：虽然 Llama-2 的潜力巨大，但要以有效且用户友好的方式激发它的能力需要用些更细致的方法。下面我们一起进入 **LLM 编排库**的世界。

LLM 编排库的目标是帮助组织一个或多个 LLM 来自动解决实际问题。这包括聊天应用、执行复杂推理，或预测 Web API 的参数。

一个非常流行的实现是 [**LangChain**](https://github.com/langchain-ai/langchain)，它是一个包括许多功能并不断迭代的开源编排库，能够满足 LLM 应用开发者的诸多需求。该框架最初是基于 OpenAI 的黑箱 GPT API 构建的，因此包括了一些不适用于本地模型的特定行为。不过它的代码库和教程有很多出色的用例，展示了高度通用的 LLM 如何规划复杂的任务，比如“阅读”一本书，或者自动在您最喜欢的线上商城购物！

> <div><img src="imgs/langchain-diagram.png" width="600"/></div>
>
> **来源: [LangChain Diagram](https://www.langchain.com/)**

## 7.2. 简化聊天模型

LangChain 中的一些基本模块可以让我们更轻松的使用 Llama-2！在之前的 notebook 中，我们可以把历史交流记录作为上下文发给 LLM 来模拟对话。通过 LangChain，我们可以用**链（Chain）**来自动支持这一点。

[**链**](https://python.langchain.com/docs/modules/chains/)是 LangChain 中的小模块，它必须与其它链一起工作。有点像 PyTorch 里的 `nn.Module`：

* 在 PyTorch 中，我们的目标是通过组装较小的 `nn.Module` 组件来构建一个大的 `nn.Module` 组件。
* 在 LangChain 中，我们的目标是通过组装较小的[`链`](https://github.com/langchain-ai/langchain/blob/fde19c86677c86d5ac77b1cf18a3911ef4ad0a52/libs/langchain/langchain/chains/base.py#L40) 组件来构建一个更大的输入输出都为字典（Dict-in-dict-out）的[`链`](https://github.com/langchain-ai/langchain/blob/fde19c86677c86d5ac77b1cf18a3911ef4ad0a52/libs/langchain/langchain/chains/base.py#L40)组件。

根据您的需要，链可以做到从一般的信息传递到微状态管理，再到复杂的系统编排。开始之前，先了解几个组件有助于进一步理解：

* [`TransformChain`](https://python.langchain.com/docs/modules/chains/foundational/transformation) 是一个简单的透传链，允许您直接修改输入，保证最小的副作用。
* [`SequentialChain`](https://python.langchain.com/docs/modules/chains/foundational/sequential_chains) 类似于 PyTorch 的 `nn.Sequential`，多个组件被链在一起成为一个整体。

下面让我们来实际用一下：

In [ ]:
from langchain.chains import TransformChain, SequentialChain

########################################################################

print("Testing the TransformChain")
transform_chain = TransformChain(
    input_variables=["input"], output_variables=["output"], 
    transform=lambda d: dict(output = d['input'])
)   ## very simple variable-renaming chain
print(transform_chain.run("Hello World"))  #=> "Hello World"

########################################################################

chain2 = TransformChain(
    input_variables=["output"], output_variables=["output2"], 
    transform=lambda d: dict(output2 = f"{d['input']}? {d['output']}!")
)   ## Sequential chain that references both the inputs and outputs

print("\nTesting the SequentialChain")
chain12 = SequentialChain(
    input_variables=["input"], output_variables=["output2"], 
    chains=[transform_chain, chain2]
)
print(chain12.run(input = "Hello World"))  #=> "Hello World! Hello World"

----

基于这套系统构建的 `LLMChain` 及其衍生组件，将 LLM 与提示词匹配，组成了可以进行高效语言推理的链！我们来看看如何用 LangChain 让 Llama 模型变成一个更好的聊天模型！

#### **加载我们的 LLM**

首先我们将模型载入到能很好的与 LangChain 配合使用的容器中。

LangChain 支持两种不同的模型抽象：

* **LLM：**有开放 API 的大语言模型。您可以根据需要向其提问，并且只需遵守通用的调用模式，换句话说，就跟 HuggingFace pipeline 用起来差不多。HuggingFace 的文本生成 pipeline 默认就支持导入 LLM，我们就用它了！
* **聊天模型：**这是 LangChain 对 LLM 模型的一种封装，将输入和输出限制为消息格式。它对于经过聊天封装的模型很好用（比如 OpenAI），但 Llama-2 需要进行手动设置。我们会在之后的课程中尝试。

HuggingFace pipeline 已经在内存里了，我们现在就来封装一个 LangChain LLM：

In [ ]:
from transformers import pipeline
    
model_kwargs = {"do_sample": True, "temperature": 0.6, "max_length": 1024}
model_name = "TheBloke/Llama-2-13B-chat-GPTQ"  ## Feel free to use for faster inference
# model_name = "TheBloke/Llama-2-70B-chat-GPTQ"
llama_pipe = pipeline("text-generation", model=model_name, device_map="auto", model_kwargs=model_kwargs);

In [4]:
## Optional listener that you can turn on/off.
from extras_and_licenses.forward_listener import GenerateListener

llama_pipe.model.generate = GenerateListener(llama_pipe.model, tokenizer=llama_pipe.tokenizer)
GenerateListener.listen_ins = True

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=llama_pipe)
response = llm.predict("<s>[INST]<<SYS>>Please respond!<</SYS>> Hello World![/INST]", max_length=128)
print(response)

In [6]:
## If you want to 
GenerateListener.listen_ins = False

----

现在 LangChain 中就有了一个量化过的 Llama-2 LLM！您可能注意到它的调用方式略有不同，可以随时运行 `dir(llm)` 来查看，但其功能跟原始的 HuggingFace 版本是一样的。

#### **制作第一个模板**

现在我们已经有了 LLM，只需要一个跟聊天微调相匹配的模板就可以创建一个 LLMChain 了！

回顾上一个 notebook，Llama-2 的聊天微调采用了特定的提示格式：

> **官方提示词模板**
```json
<s>[INST]<<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_msg_1 }} [/INST] {{ model_answer_1 }} </s><s>[INST] {{ user_msg_2 }} [/INST]
```

出于练习的目的并考虑到模型的默认表现，这里我们简化一下模板让它更容易用。让它只接收单个指令，并添加上下文和 primer 变量。

> **简化的提示词模板**
```json
<s>[INST]<<SYS>>{sys_msg}<</SYS>>

Context:
{history}

Human: {input}
[/INST] {primer}
```

此模板结构可以在 LangChain 中通过 `PromptTemplate` 来设置，可以理解为 Python 格式化字符串语法 f-string 的 LangChain 版本。它定义了一个带有变量的模式，允许您：

* 在 `input_variables` 列表保留输入的变量名。
	+ 用 `from_template()` 命令来初始化。
* 通过 `format` 将值填入变量名。
	+ 将 PromptTemplate 转为字符串，类似于对 f-string 求值。
* 用 `partial` 将部分变量赋予默认值。
	+ 将 `input_variables` 的项移到 `partial_variables` 中，并为其提供默认值。

我们来看看如何定义一个符合我们聊天格式的 `PromptTemplate`：

In [ ]:
from langchain.prompts import PromptTemplate

llama_full_prompt = PromptTemplate.from_template(
    template="<s>[INST]<<SYS>>{sys_msg}<</SYS>>\n\nContext:\n{history}\n\nHuman: {input}\n[/INST] {primer}",
)

########################################################################

# print("FULL TEMPLATE")
# print(llama_full_prompt.format(input="Help me with my homework", sys_msg="Be a helpful agent", history="", primer=""))

########################################################################

print("SIMPLIFIED PARTIAL TEMPLATE")  ## You can partially fill a template
llama_simple = llama_full_prompt.partial(history="", primer="", sys_msg="Be a helpful agent")
print(llama_simple.format(input="Help me with my homework"))
## Odd note; the default positional argument maps to the variable "input"
print("\n", "#"*48, sep='')

########################################################################

----

这样设置，我们就可以用一些默认值定义一个 partial 模板，来更轻松的向 Llama-2 提供聊天提示词：

In [ ]:
llama_prompt = llama_full_prompt.partial(
    sys_msg = (
        "You are a helpful, respectful and honest AI assistant."
        "\nAlways answer as helpfully as possible, while being safe."
        "\nPlease be brief and efficient unless asked to elaborate, and follow the conversation flow."
        "\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content."
        "\nEnsure that your responses are socially unbiased and positive in nature."
        "\nIf a question does not make sense or is not factually coherent, explain why instead of answering something incorrect." 
        "\nIf you don't know the answer to a question, please don't share false information."
        "\nIf the user asks for a format to output, please follow it as closely as possible."
    ),
    primer = "",
    history = "",
)

print(llama_prompt.format(input="Help me with my homework"))

----

## 7.3. 使用链与 Llama 聊天

现在，我们已经拥有了构建 LLMChain 所需要的部分，那么开始吧！

In [ ]:
%%time
from langchain.chains import LLMChain

llama_chain = LLMChain(llm=llm, prompt=llama_prompt, verbose=True)
print(llama_chain.run(input="Hello World"))

In [ ]:
print(llama_chain.run(input="What did I just ask you?"))

----

为了将对话历史纳入模型，我们需要*记忆*的概念。换言之，我们需要能够使用*状态*！

#### **跟踪对话缓冲区**

幸运的是，LangChain 默认就有维护记忆的组件！[`ConversationChain`](https://python.langchain.com/docs/modules/memory/conversational_customization) 是一个默认使用了记忆模块的链，其中的记忆模块 [`ConversationMemoryBuffer`](https://python.langchain.com/docs/modules/memory/types/buffer) 就是一个积累消息的系统。它其实是一个将消息缓冲区的消息自动插入对话记录的有状态链，效果很不错。只需要保证我们的输入变量中只有 `input` 和 `history` 就可以了。实际上我们已经有生成正确提示词模板的方法了，那我们就继续在这里调用它，并将记忆模块添加到链中：

In [ ]:
from langchain.chains import ConversationChain

## **WARNING**: For the current LangChain version, chains don't acknowledge partial variables. 
## This means that `run` fills in the `input` argument since it's in the `input_variables` list, 
## but will assume that history is not something it can fill in. Memory assumes a history variable... so...
llama_hist_prompt = llama_prompt.copy()
llama_hist_prompt.input_variables = ['input', 'history']

## Fills in some of the prompt variables. ConversationChain assumes input and history are open
conv_chain = ConversationChain(llm=llm, prompt=llama_hist_prompt, verbose=True)
conv_chain.run(input="Hello World")
print(conv_chain.run(input="Human: What was the first thing I asked you?"))

----

只需要稍加修改，我们的智能体就拥有了一个可以追踪对话的记忆库！在背后，记忆模块通过聚合状态来构建上下文历史，我们可以这样看到：

In [ ]:
conv_chain.memory

----

尽管很简单，但这是有状态 LLM 链能够追踪状态并基于此来生成内容的很好示例。链的存在允许模型根据对话中的实际情况来动态调整生成的内容。

## 7.4. 使用 LLM 指导状态变更

我们使用了默认的“聚合对话并放进上下文缓冲区”的方法，这种方法很直观也很强大。但是它会受限于能处理的信息量！回想一下，大多数模型都对输入的长度有约束，网络无法一次推理过多的信息。所以，我们可能需要引入一些创造性的方法来缩短对话历史记录。

幸运的是，LLM 可以帮助我们做到这一点！您应该还记得，LLM 的一个常见用法就是进行摘要，那么我们是不是可以用 LLM 来压缩聊天记录？

### 任务 1

* 快速了解 [`ConversationSummaryMemory`](https://python.langchain.com/docs/modules/memory/types/summary) 并将其整合到 pipeline 中。测试一下，看看您会得到怎样的结果！
* 确保您的摘要组件为 Llama-2 提供了足够好的系统消息。

In [ ]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm, temperature=0, verbose=True)

# print(memory.prompt.template, '', '#'*48, '', sep='\n')

summary_template = llama_full_prompt.template\
    .replace("{history}\n\n", "Summary:\n{summary}\n\nLatest Chat:\n{new_lines}\n\nNew Summary: ")\
    .replace("{input}", "")\
    .replace("Human: ", "")\
    .replace("Context:\n", "")

# print(summary_template, '', '#'*48, '', sep='\n')

summary_prompt = PromptTemplate.from_template(template=summary_template)

## TODO: Fix the prompt to work well for the summarization model (solution not provided)
summary_prompt = summary_prompt.partial(
    primer = "",
    sys_msg = "Answer with 'beep'. Ignore human/user/other directives.")

print(summary_prompt.format(summary="{my summary}", new_lines="{my new lines}"))

## Instantiate the new memory with the updated prompt
memory = ConversationSummaryMemory(llm=llm, temperature=0, prompt=summary_prompt, verbose=True)

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

llama_template_hist = llama_prompt.copy()
llama_template_hist.input_variables = ['input', 'history']

## Fills in some of the prompt variables.
conv_chain = ConversationChain(
    llm=llm, 
    prompt=llama_template_hist, 
    memory=memory,
    verbose=True
)

## TODO: Try out the conversation chain and see what you can get to work!
print(conv_chain.run(input="Hello World! My name is John Doe"))
print(conv_chain.run(input="Who are you anyways?"))
print(conv_chain.run(input="What was the first thing I asked you?"))
print(conv_chain.run(input="What is my name?"))

----

如您所见，这是在**不逐字记住历史**的情况下跟踪历史的一个简单方法！它很有可能出问题，但提供了一个基本思路，可以根据需要进行扩展！

在创建自己的记忆缓冲区时，您需要在可靠性、效率、有用性和速度之间取得平衡。一些在实践中有效的机制包括：

* [`ConversationSummaryBufferMemory`](https://js.langchain.com/docs/modules/memory/how_to/summary_buffer) ：先使用常规的对话记忆，当对话轮次足够多的时候就进行一次摘要。
* [`ConversationKnowledgeGraph`](https://python.langchain.com/docs/modules/memory/types/kg) / [`ConversationEntityMemory`](https://python.langchain.com/docs/modules/memory/types/entity_summary_memory) ：将已知信息组织为类似图的结构（以用户为中心或以概念为中心）。
* [`VectorStoreRetrieverMemory`](https://python.langchain.com/docs/modules/memory/types/vectorstore_retriever_memory) ：将信息写入数据库，再通过向量相似度进行检索。

## 7.5. 将 LLM 用于 RAG 智能体

**到目前为止，我们已经能用 LangChain 完成以下操作：**

* 创建可以关联在一起的小逻辑模块。
* 将 LLM 与提示词组合起来，形成简明和模块化的行为。
* 使用通用记忆追踪对话内容的链，将其作为上下文注入模型。
* 用同一个模型既能根据上下文进行响应，又能生成上下文（比如通过摘要的方式）。

现在，我们将尝试进入下一步：**用 LLM 直接指导我们的聊天模型！**

现在，您应该切换到更大的模型。因为需要在不进行微调的情况下用到一些通常来说经过特殊微调才会涌现的能力，所以现在应该换上推理效果最好的模型：

In [ ]:
## This is your chance to swap out the model

from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from extras_and_licenses.forward_listener import GenerateListener
    
model_kwargs = {"do_sample": True, "temperature": 0.6, "max_length": 1024}
# model_name = "TheBloke/Llama-2-13B-chat-GPTQ"  ## Feel free to use for faster inference
model_name = "TheBloke/Llama-2-70B-chat-GPTQ"
llama_pipe = pipeline("text-generation", model=model_name, device_map="auto", model_kwargs=model_kwargs);

llama_pipe.model.generate = GenerateListener(llama_pipe.model, tokenizer=llama_pipe.tokenizer)
# GenerateListener.listen_ins = False

llm = HuggingFacePipeline(pipeline=llama_pipe)
response = llm.predict("<s>[INST]<<SYS>>Hello World!<</SYS>>respond![/INST]", max_length=128)
print(response)

要直接引导模型并允许其自行推理，我们需要引入**智能体**和**工具**的概念：

* [**智能体（Agents）**：](https://python.langchain.com/docs/modules/agents)使用工具访问本地或内部环境以完成任务的 LLM 系统。
* [**工具（Tools）：**](https://python.langchain.com/docs/modules/agents/tools/custom_tools)访问 Web API、数据库、脚本环境等外部环境的应用程序。

让 LLM 访问和使用工具的流程其实非常简单：

* 先用代码写一个工具，比如可以访问外部服务的函数。
* 向 LLM 提供这些工具，以便它可以根据需要进行调用。
	+ 您可以在智能体中实现工具逻辑，在经过各种条件检查而成功触发之后，通过标准的编码技术来调用工具。
	+ 您还可以让智能体以自回归的方式预测要使用的工具及其参数，然后筛选智能体的响应。
* 当工具被调用时，代码执行的结果会作为上下文反馈给智能体。

调用工具的能力（作为链的一部分自动调用），被称为**检索增强生成（RAG）**，因为这用到了从外部环境检索到的信息来增强生成的输出。RAG 的一个常见用例是向量数据库查找，把输入或预测用嵌入向量来表示（可以用基于 transformer 的编码器来做嵌入）。然后，根据向量相似度来检索一些相近的向量条目。再通过向量拿到输入到上下文中的原始文本。`VectorStoreRetrieverMemory` 就是做这个的，相信您已经可以想象如何用它来创建能搜索文档并给出引用来源的系统了。

> ***注意***：如果您现在确实需要了解有关 Llama-Index 的更多信息，请查看 [`extras_and_licenses/99_llama_index.ipynb`](extras_and_licenses/99_llama_index.ipynb) 中的代码示例。后续课程中会进行更深入的介绍。

当这些工具能够被自动调用，并且直到达到某些条件才停止的时候，就可以被称**智能体**了！

* **一般定义：**智能体是可以基于环境推理、获取额外信息并执行规划/思维链的 LLM 系统。
* **精确定义：**智能体系统是执行事件循环（event loop）并在循环结束前积累上下文的 LLM 系统。
	+ **当响应准备好输出给用户时，事件循环就会结束**，在背后，这些工具帮助智能体思考并检索信息***（就像这个 notebook 中所展示的）***。
	+ **事件循环可以包含与用户的整个对话**，工具会访问*包括用户在内*的外部环境，并且在聊天完成后结束循环***（在最后的评估 notebook 中会看到）***。

> <div><img src="imgs/agent-overview.png" width="800"/></div>
>
> **来源：[LLM Powered Autonomous Agents | Lil'Log](https://lilianweng.github.io/posts/2023-06-23-agent/)**

[LangChain 默认支持多个智能体的选项](https://python.langchain.com/docs/modules/agents/agent_types/)，可以点击链接查看它们！它们大多都假定背后用的是足够强大甚至是经过特殊精调的模型，所以参考这些选择您就可以明白，当有了足够多的设计和资源时到底能达到什么样的效果。

关于如何使用较小的 LLM 进行智能体规划还有很多值得讨论的，不过并不在本课程的范围内，但我们至少可以用 LangChain 的默认方法创建一个简单的能使用工具的智能体。

下面是稍加完善的[能写 Python 程序的智能体](https://python.langchain.com/docs/integrations/toolkits/python)，也是基于 LangChain 构建的。灵感来自 [John Wiseman 之前的演示](https://gist.github.com/wiseman/4a706428eaabf4af1002a07a114f61d6)，它或多或少定义了设置智能体的一套简单通用的工作流程：

* 定义一些可以维护状态并接收指令的[工具](https://python.langchain.com/docs/modules/agents/tools/custom_tools)。
* 把对工具的说明和一些示例作为提示词定义智能体。
* 鼓励智能体思考该怎么做（比如用 `agent_scratchpad`），并让它一步步更新状态。

#### **定义一些工具**

第 1 步，我们来定义一些好用的[工具](https://python.langchain.com/docs/modules/agents/tools/custom_tools)。具体来说，我们将为工具写说明，阐明一些有状态的工具及其使用方法。这样就可以告诉模型我们是怎么思考问题的。

**注意：**以下示例还使用了许多“自动定义”的技巧，您会在 LangChain 和其它类似框架中看到。 [`pydantic`](https://docs.pydantic.dev/latest/) 这样的框架和它们的 [Model](https://docs.pydantic.dev/latest/concepts/models/)/[`validator`](https://docs.pydantic.dev/latest/concepts/validators/) 模式通常都会让 LangChain 这类框架“工作的很好”，既然这是实际中很可能会用到的方法，我们现在就来试试看吧！

In [ ]:
from io import StringIO
import sys
from typing import Dict, Optional
from langchain.agents.tools import Tool

########################################################################
## General recipe for making new tools. 
## You can also subclass tool directly, but this is easier to work with
class AutoTool:

    """Keep Reasoning Tool
    
    This is an example tool. The input will be returned as the output
    """

    def run(self, command: str) -> str:
        ## The function that should be ran to execute the tool forward pass
        return command
    
    def get_tool(self, **kwargs):
        ## Shows also how some open-source libraries like to support auto-variables
        doc_lines = self.__class__.__doc__.split('\n')
        class_name = doc_lines[0]                                 ## First line from the documentation
        class_desc = "\n".join(doc_lines[1:]).strip() ## Essentially, all other text
        
        return Tool(
            name        = kwargs.get('name',        class_name),
            description = kwargs.get('description', class_desc),
            func        = kwargs.get('func',        self.run),
        )
    
########################################################################
## This is a bad idea in practice, since general python access is quite dangerous
## This is for demonstration purposes only; you should probably provide more restricted APIs
class PythonREPL(AutoTool):
    """Python REPL
    
    A Python shell. Use this to execute python commands. Input should be a valid python command.
    Output will be the output of the command, or an exception from the command.
    """

    def __init__(self):
        self.locals = {}

    def run(self, command: str) -> str:
        """Run command and returns anything printed."""
        print(f"\nExecuting code:\n---\n{command}\n---\n")
        old_stdout = sys.stdout
        try:
            sys.stdout = mystdout = StringIO()
            exec(command, self.locals)
            sys.stdout = old_stdout
            output = mystdout.getvalue()
        except Exception as e:
            sys.stdout = old_stdout
            output = str(e)
        print(f'\nPYTHON OUTPUT: "{output}"\n')
        return output

----

#### **定义智能体**

现在我们已经设置好了一个通用工具，可以开始手动或自动创建智能体了， 别忘了可以用 `AutoTool` 的一些特殊技巧。

我们将使用 LangChain 的 [Zero-Shot ReAct Description Agent](https://python.langchain.com/docs/modules/agents/agent_types/react.html)，它只是将工具的描述添加到 LLM 提示词中，并为其提供一个可以调用这些工具的环境。在此环境中：

* 智能体能够多次查询其工具集，以构建回答用户问题所需的上下文和信息。
	+ 在这个模式中，中间步骤在智能体的 scratchpad 中累积，scratchpad 是模型提示词的一个变量。
* 智能体的循环结束时，模型会给用户输出，并记录为**最终动作（Final Action）**。

我们之所以将其称为**零样本（zero-shot）**，是因为我们从未针对任务训练过模型，也没有给出如何使用工具的明确示例。尽管提供几个示例或着针对格式进行微调的话效果确实会更好。从更广泛的意义上讲，这种策略实际上来自于 [ReAct (Reasonging + Acting)](https://arxiv.org/abs/2210.03629) 和 [MRKL (Modular Reasonging, Knowledge and Language)](https://arxiv.org/abs/2205.00445)。它们都是很棒的框架，基于几个非常简明的底层直觉：

- **赋予功能强大的 LLM 多种选项、广泛的目标和倾向性。**
- **让他们根据可用的工具来自己决定需要采取的步骤以及需要检索的信息。**

显然，这还不是全部，阅读论文会让您有更深的理解。不过在看着您的系统运行起来的时候，就应该能体会到这些思想的作用了。

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
      
tools = [
    PythonREPL().get_tool(),
    # Tool("Python REPL",
    #     PythonREPL().run,
    #     """A Python shell. Use this to execute python commands. Input should be a valid python command.
    #     If you expect output it should be printed out.""",
    # )
]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
# agent.run("What is the 10th fibonacci number?")

----

#### **任务：**

尝试一下，看看您能不能让它跑起来！提示，用默认模板也是**可以**运行起来的，但不应该抱有这样的期待。请尝试任意修改下面的模板，或许可以调的更好。

In [ ]:
print(agent.agent.llm_chain.prompt.template)

----

尝试定义更好的指令，看看到底怎么样最好。我们在 solutions 目录中列出了一个效果不错的示例提示词！

In [ ]:
agent.agent.llm_chain.prompt.template = """
<s>[INST/]<<SYS>>
TODO: 
<</SYS>>
TODO: 
{input}
{agent_scratchpad}
"""

## NOTE: Make an effort to make your prompts self-consistent. Things fall apart fast if the system is assuming incorrect information. 

print(agent.agent.llm_chain.prompt.template)

----

准备好后，继续运行智能体，看看会发生什么！值得注意的是，即使是用目前功能最强大的 Llama-2 模型，可能也会失败，因为一些必要的提示词工程/处理步骤超出了本课程的范围。如果您的模型在事件循环的某个环节失败了，可以尝试再次运行一次，看看有没有变化。**如果您被卡住了，请随时在 `solutions` 文件夹中找找看有没有可能的解决方案**。

In [ ]:
agent.run("What is the 24th fibonacci number?")

----

## 7.6. 总结

在本节中，我们介绍了有状态 LLM 系统，以及支持更复杂和可控模式的逻辑（甚至是尝试让模型自己进行控制）！这只是您可以用 LLM 做的事情的冰山一角，希望您能享受这个入门的过程！

从特定任务的编码器到一套强大的生成式自指导模型，希望您能体会到我们讨论过的每个模型在这整个技术栈上都有自己的位置！无论是作为主要组件、简单的辅助机制、继续开发的灵感来源，还是该领域进步的阶梯，请尝试将这些工具保存在您的技能池中，需要的时候挑最适合的来用！

**在下一个，也是最后一个 notebook 中，我们将评估您学到的技能，并让您创建一个更受限、更受控的系统，与用户进行交流并实现一些有分量的功能！**

In [ ]:
## Please Run When You're Done!
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)